In [ ]:
import os

import pandas as pd
import numpy as np

import unicodedata

In [ ]:
def remove_control_characters(s):
    return "".join(ch for ch in s if unicodedata.category(ch)[0]!="C")

In [ ]:
def parse_xls(source_dir):
    all_xls_filepaths = [os.path.join(root, file) 
                         for root, dirs, files in os.walk(source_dir)
                         for file in files
                         if os.path.splitext(file)[1] == '.xls']
    
    df = pd.DataFrame()

    for xls_file in all_xls_filepaths:
        xls = pd.ExcelFile(xls_file)
        for sheet in xls.sheet_names:
            new_df = xls.parse(sheet)
            new_df = new_df.replace(np.nan, '', regex=True)
            df = df.append(new_df)
            
        
    return df


In [ ]:
aaron_edits_dir = '/home/francis/Desktop/CurrentProjects/CWBRuploadRush/AaronsEdits/Mar5CwbrwithTitle/'
aaron_df = parse_xls(aaron_edits_dir)

In [ ]:
aaron_df.size

In [ ]:
tom_edits_dir = '/home/francis/Desktop/CurrentProjects/CWBRuploadRush/Mar5CwbrwithTitle/'
tom_df = parse_xls(tom_edits_dir)

In [ ]:
tom_df.size

In [ ]:
whittle_df = pd.DataFrame.copy(tom_df)

In [ ]:
mismatches = []

for book_id in aaron_df['book_id']:

    aaron_row = aaron_df.loc[aaron_df['book_id'] == book_id]
    aaron_row_dict = aaron_row.to_dict()
    
    tom_row = tom_df.loc[tom_df['book_id'] == book_id]
    tom_row_dict = tom_row.to_dict()
    
    for k, v in aaron_row_dict.items():
        if tom_row_dict[k] == v:
            continue
        else:
            mismatches.append((book_id, aaron_row_dict['season'], aaron_row_dict['publication_date'], aaron_row_dict, tom_row_dict))
            break

for book_id, season, pub_data, aaron_dict, tom_dict in mismatches:
    print("\n**{}**".format((book_id, season, pub_data)))
    for k, v in aaron_dict.items():
        if tom_dict[k] == v:
            continue
        print('aaron: {}\ntom: {}\n'.format(v, tom_dict[k]))